In [1]:
#!/usr/bin/env python3

import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

from load_mnist import load_mnist

import freeze_tf2_model

2023-11-14 16:07:13.745032: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-14 16:07:13.795690: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 16:07:13.795765: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 16:07:13.795845: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 16:07:13.811351: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-14 16:07:13.812331: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
# remove if you have a GPU/CUDA
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

print ("Numpy version:     ", np.__version__)
print ("TensorFlow version:", tf.__version__)

Numpy version:      1.26.1
TensorFlow version: 2.14.0


In [3]:
#---------------------------------------------------------------------------------
#  Load and pre-process MNIST dataset
#---------------------------------------------------------------------------------
batch_size      = 100
epochs          = 5
num_test_images = 10
tftype = tf.float32
nptype = np.float32

(train_eval_data, train_eval_labels), (test_data, test_labels) = load_mnist(datatype=nptype)

train_eval_labels = np.asarray (train_eval_labels, dtype=nptype)
train_eval_labels = to_categorical (train_eval_labels)

test_labels = np.asarray (test_labels,  dtype=nptype)
test_labels = to_categorical (test_labels)

#  Split the training data into two parts, training and evaluation
data_split   = np.split (train_eval_data,   [55000])
labels_split = np.split (train_eval_labels, [55000])

train_data   = data_split [0]
train_labels = labels_split [0]

eval_data    = data_split [1]
eval_labels  = labels_split [1]

print ('Size of train_data: ', train_data.shape [0])
print ('Size of eval_data: ',  eval_data.shape [0])
print ('Size of test_data: ',  test_data.shape [0])

train_data = train_data.reshape ((55000, 28,28,1))
eval_data  = eval_data.reshape  ((5000,  28,28,1))
test_data  = test_data.reshape  ((10000, 28,28,1))

test_data   = test_data [:num_test_images]
test_labels = test_labels [:num_test_images]

Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Size of train_data:  55000
Size of eval_data:  5000
Size of test_data:  10000


In [4]:
#---------------------------------------------------------------------------------
#  Create and train model
#---------------------------------------------------------------------------------
inputs = keras.Input(shape=(28, 28, 1), name="image")
x = layers.Conv2D(32, 5, activation="relu")(inputs)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, 5, activation="relu")(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Flatten()(x)
x = layers.Dense(10, activation='softmax', name='OUTPUT')(x)
tfmodel = keras.Model(inputs, x)

tfmodel.compile (loss='categorical_crossentropy',
                 optimizer='sgd',
                 metrics=['accuracy'])

tfmodel.fit (train_data, train_labels,
             batch_size = batch_size,
             epochs = epochs,
             verbose = 1
            )

tfmodel.summary ()

Epoch 1/5
550/550 [==============================] - 14s 25ms/step - loss: 0.9741 - accuracy: 0.7503
Epoch 2/5
550/550 [==============================] - 12s 22ms/step - loss: 0.2741 - accuracy: 0.9204
Epoch 3/5
550/550 [==============================] - 11s 20ms/step - loss: 0.1992 - accuracy: 0.9420
Epoch 4/5
550/550 [==============================] - 13s 23ms/step - loss: 0.1589 - accuracy: 0.9538
Epoch 5/5
550/550 [==============================] - 12s 22ms/step - loss: 0.1345 - accuracy: 0.9604
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)               

In [5]:
#---------------------------------------------------------------------------------
#  Evaluate Model
#---------------------------------------------------------------------------------
loss, acc = tfmodel.evaluate(eval_data, eval_labels)
print ('Loss: ', loss, '  Accuracy: ', acc)

157/157 [==============================] - 1s 3ms/step - loss: 0.0978 - accuracy: 0.9728
Loss:  0.09775587171316147   Accuracy:  0.9728000164031982


In [7]:
#-------------------------------------------------------------------------
#   Freeze and save the model
#-------------------------------------------------------------------------
# The model will be saved to my_frozen_model/ and will input/output primitive Vertica columns
freeze_tf2_model.freeze_model(tfmodel, './my_frozen_model', '0')

Saving frozen model to: ./my_frozen_model1/frozen_graph.pb
Saving model description file to: ./my_frozen_model1/tf_model_desc.json


2023-11-14 16:08:20.701612: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-11-14 16:08:20.701924: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
